# 5 - Tool Use
- LangChain provides a straightforward way to bind tools to an LLM, allowing the LLM to decide when to generate a tool call.
- For this example, we will define a simple Python function that simulates looking up information (e.g., a fake search tool) and then create a LangChain agent that can use this tool based on user input.


In [8]:
import os
import asyncio
from typing import List, Dict, Any

# from langchain_openai import ChatOpenAI # Commented out to default to Google
from langchain_google_genai import ChatGoogleGenerativeAI # Uncommented for Google Gemini
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool # Decorator to easily define tools
from langchain.agents import create_tool_calling_agent, AgentExecutor # Components for creating a tool-calling agent

In [9]:
import os
from dotenv import load_dotenv
from google import genai


# Load environment variables from .env file
load_dotenv()
# Using Google GenAI
# Verify if API key is present
if not os.getenv('GOOGLE_API_KEY'):
    raise ValueError("Google API key not found. Please check your .env file.")
else:
    client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))
    response = client.models.generate_content(
        model="gemini-2.0-flash", contents="Write a random small 1 line Thought of the day!"
        # model="gemini-2.5-pro-preview-05-06", contents="Write a random small 1 line Thought of the day!"
    )
    print(response.text)

Embrace the glorious mess that you are.



In [10]:
# Using LangChain Google GenAI
# Initialize the language model
# A model with function calling capabilities is required for this example.
try:
    # Example for OpenAI (gpt-4o-mini is a good choice for cost/capability)
    # llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) # Commented out
    
    # Example for Google (uncomment and replace if using Google)
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro-preview-05-06", temperature=0) # Gemini models support function calling
    print(f"Language model initialized: {llm.model}")

    # Create a simple prompt
    prompt = ChatPromptTemplate.from_messages([
        ("user", "What's Agentic AI in brief?")
    ])
    
    # Create a simple chain
    chain = prompt | llm | StrOutputParser()
    
    # Get the response
    response = chain.invoke({})
    print("\nAI Response:")
    print(response)  


except Exception as e:
    print(f"Error initializing language model: {e}")
    print("Please ensure your LLM supports function calling and your API key is valid.")
    llm = None # Set llm to None if initialization fails

Language model initialized: models/gemini-2.5-pro-preview-05-06

AI Response:
In brief, **Agentic AI** refers to AI systems that can **proactively and autonomously perceive their environment, make decisions, and take actions to achieve specific goals.**

Think of them as AI "doers" or "agents" that don't just respond to prompts, but can:
*   **Set sub-goals**
*   **Plan sequences of actions**
*   **Use tools (like code interpreters, web browsers, APIs)**
*   **Learn from feedback**
*   **Operate with a degree of independence**

They aim to complete complex tasks with minimal human intervention.


In [11]:
# --- Define a Tool ---
# We use the @tool decorator to easily define a function as a tool.
# The docstring is used as the tool's description, which the LLM sees.
# The function signature defines the parameters the tool accepts.

@tool
def search_information(query: str) -> str:
    """
    Searches for information on a given query.
    Use this tool to find answers to factual questions or get information about topics.
    Provides a simulated search result.
    """
    print(f"\n--- Tool Called: search_information with query: '{query}' ---")
    # Simulate searching for information
    simulated_results = {
        "weather in London": "The weather in London is currently cloudy with a temperature of 15°C.",
        "capital of France": "The capital of France is Paris.",
        "population of Earth": "The estimated population of Earth is around 8 billion people.",
        "tallest mountain": "Mount Everest is the tallest mountain above sea level.",
        "default": f"Simulated search result for '{query}': Information found about {query}."
    }
    # Return a simulated result based on the query
    result = simulated_results.get(query.lower(), simulated_results["default"])
    print(f"--- Tool Result: {result} ---")
    return result

# List of tools available to the agent
tools = [search_information]

In [12]:
# --- Create a Tool-Calling Agent ---
# LangChain provides a convenient way to create agents that are capable of using tools.
# create_tool_calling_agent is suitable for models with native function calling.

# Define the prompt for the agent. It needs a placeholder for tool instructions and chat history.
# The `tools_prompt` template is automatically populated by LangChain with tool definitions.
# The `agent_scratchpad` is where the agent writes its thoughts and tool calls.
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions using the available tools."),
    ("placeholder", "{chat_history}"), # For conversational memory if needed
    ("human", "{input}"), # The user's current input
    ("placeholder", "{agent_scratchpad}"), # Where the agent plans and calls tools
])

In [14]:
# Run the example with a test topic
# pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


# Create the agent
if llm: # Only create agent if LLM initialized successfully
    agent = create_tool_calling_agent(llm, tools, agent_prompt)

    # Create an AgentExecutor to run the agent.
    # AgentExecutor is the runtime that invokes the agent and executes tool calls.
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # verbose=True shows the agent's thought process


    # --- Run the Agent ---
    async def run_agent_with_tool(query: str):
        """Runs the agent with a given query."""
        print(f"\n--- Running Agent with Query: '{query}' ---")
        try:
            # Invoke the agent executor asynchronously
            response = await agent_executor.ainvoke({"input": query, "chat_history": []}) # Pass empty chat history for this example
            print("\n--- Final Agent Response ---")
            print(response["output"]) # The final answer from the agent
        except Exception as e:
            print(f"\nAn error occurred during agent execution: {e}")
            print("Please ensure your LLM supports function calling and your API key is valid.")


    # Test the agent with queries that should trigger the tool
    async def main():
        # Run the asynchronous function with different queries
        # asyncio.run(run_agent_with_tool("What is the capital of France?"))
        # asyncio.run(run_agent_with_tool("Tell me about the population of Earth."))
        # asyncio.run(run_agent_with_tool("What's the weather like in London?"))
        asyncio.run(run_agent_with_tool("What is the tallest mountain?"))
        asyncio.run(run_agent_with_tool("Tell me something about a sausage dog in 2 sentences.")) # Should use the default simulated result

else:
    print("\nSkipping agent execution due to LLM initialization failure.")

# Run the async function
await main()


--- Running Agent with Query: 'What is the tallest mountain?' ---


> Entering new AgentExecutor chain...

Invoking: `search_information` with `{'query': 'tallest mountain'}`



--- Tool Called: search_information with query: 'tallest mountain' ---
--- Tool Result: Mount Everest is the tallest mountain above sea level. ---
Mount Everest is the tallest mountain above sea level.Mount Everest is the tallest mountain above sea level.

> Finished chain.

--- Final Agent Response ---
Mount Everest is the tallest mountain above sea level.

--- Running Agent with Query: 'Tell me something about a sausage dog in 2 sentences.' ---


> Entering new AgentExecutor chain...

Invoking: `search_information` with `{'query': 'sausage dog'}`



--- Tool Called: search_information with query: 'sausage dog' ---
--- Tool Result: Simulated search result for 'sausage dog': Information found about sausage dog. ---
Simulated search result for 'sausage dog': Information found about sausage dog.Sausage dogs, als